Using openai batch processing to run fc metric on a large dataset
This is cheaper and the laptop does not have to be on all the time.

In [20]:
import json

from openai import OpenAI
import os
import pandas as pd
from fc_metric import calculate_fc_using_gpt

In [32]:
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "Please set the OPENAI_API_KEY environment variable before running the factual consistency metric script"
    )
client = OpenAI()

In [33]:
abstracts_file_name = 'scientific_nature_DOIs'

In [34]:
df = pd.read_csv(f"../data/identifiers_and_abstracts/{abstracts_file_name}.csv")
df.dropna(subset=["sem_scholar_abstract"], inplace=True, ignore_index=True)
df

,id,category,title,url,identifier,sem_scholar_title,sem_scholar_abstract
0,815vqz,scientific,Signal detected from the first stars in the un...,https://www.nature.com/articles/nature25791,DOI:10.1038/nature25791,Possible interaction between baryons and dark-...,The cosmic radio-frequency spectrum is expecte...
1,81ymjh,scientific,Artificial intelligence techniques able to lea...,https://www.nature.com/articles/s41567-018-0048-5,DOI:10.1038/s41567-018-0048-5,Neural-network quantum state tomography,The experimental realization of increasingly c...
2,82bpf1,scientific,Thrombin@Fe 3 O 4 nanoparticles for use as a h...,https://www.nature.com/articles/s41598-017-186...,DOI:10.1038/s41598-017-18665-4,Thrombin@Fe3O4 nanoparticles for use as a hemo...,Bleeding remains one of the main causes of pre...
3,82dzyr,scientific,Scientists have found a breakthrough technique...,https://www.nature.com/articles/s41557-018-0009-8,DOI:10.1038/s41557-018-0009-8,Control over phase separation and nucleation u...,Control over the nucleation of new phases is h...
4,82es30,scientific,Simulating Uneven Trails Provides Insight Into...,https://www.nature.com/articles/s41598-018-210...,DOI:10.1038/s41598-018-21018-4,Challenging human locomotion: stability and mo...,The need to move over uneven terrain is a dail...
...,...,...,...,...,...,...,...
6813,dph7bi,scientific,Neoceroplatus betaryiensisnov. sp. (Diptera: K...,https://www.nature.com/articles/s41598-019-477...,DOI:10.1038/s41598-019-47753-w,Neoceroplatus betaryiensis nov. sp. (Diptera: ...,Blue shining fungus gnats (Diptera) had been l...
6814,dpi8uf,scientific,"In this mouse model of epilepsy, the temperatu...",https://www.nature.com/articles/s41374-019-0335-5,DOI:10.1038/s41374-019-0335-5,Temperature elevation in epileptogenic foci ex...,Physiological brain temperature is an importan...
6815,dppdhj,scientific,"Based on brain scans data from 1,774 people wi...",https://www.nature.com/articles/s41467-019-130...,DOI:10.1038/s41467-019-13005-8,Altered structural brain asymmetry in autism s...,Altered structural brain asymmetry in autism s...
6816,dpstjb,scientific,Researchers demonstrate generation of 2D &amp;...,https://www.nature.com/articles/s41377-019-0201-7,DOI:10.1038/s41377-019-0201-7,Dielectric metasurfaces for complete and indep...,Metasurfaces are optically thin metamaterials ...


In [35]:
df['fc_score'] = -1
if not os.path.exists(f"{abstracts_file_name}_fc_scores.csv"):
    df_out = df.copy()
else:
    df_out = pd.read_csv(f"{abstracts_file_name}_fc_scores.csv")

## Creating a batch request for the factual consistency metric using the lines from min to max

In [28]:
min = 0
max = 3000
file_name = f"request_abstracts_{abstracts_file_name}_{min}_{max}.jsonl"
with open(file_name, "w+") as f:
    for i, row in df.iterrows():
        if i < min:
            continue
        if i >= max:
            break
        messages = calculate_fc_using_gpt(row["sem_scholar_abstract"], row["title"], row["sem_scholar_title"], False, return_prompt=True)
        input = {
            "custom_id": "row-" + str(i) + "-fc-id-" + str(row["id"]),
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini",
                "messages": messages
            }
        }
        f.write(json.dumps(input) + "\n")

In [29]:
# Uploading a request file to OpenAI
request_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)
request_file

FileObject(id='file-6qBtq8dnbE2fmrsH8HVR7G', bytes=4890289, created_at=1733087645, filename='request_abstracts_scientific_DOIs_0_3000.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [30]:
# Creating a batch request using the uploaded input file
batch = client.batches.create(
  input_file_id=request_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)
batch

Batch(id='batch_674cd1a00c3c819094b7b752bc3c2dfb', completion_window='24h', created_at=1733087648, endpoint='/v1/chat/completions', input_file_id='file-6qBtq8dnbE2fmrsH8HVR7G', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1733174048, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

## Retrieving the batch to find out about the completion status

In [19]:
batch = client.batches.retrieve(batch.id)
batch

Batch(id='batch_674cd0728f788190a60e7438d277162a', completion_window='24h', created_at=1733087346, endpoint='/v1/chat/completions', input_file_id='file-JMVY314E73zFt8k2DNvDcy', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1733173746, failed_at=None, finalizing_at=None, in_progress_at=1733087347, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=571))

## Downloading the output file to get the factual consistency scores and adding them to the output_dataframe

In [92]:
response_content = client.files.content(batch.output_file_id)

In [93]:
for line in response_content.iter_lines():
    output = json.loads(line)
    score_json = output['response']["body"]["choices"][0]["message"]["content"]
    score = json.loads(score_json).get("score", -2)
    custom_id = output["custom_id"]
    row_id = int(custom_id.split("-")[1])
    post_id = custom_id.split("-")[4]
    if df.loc[row_id, "id"] != post_id:
        raise ValueError(f"Row ID {row_id} does not match post ID {post_id}")
    df_out.loc[row_id, 'fc_score'] = score
df_out

,id,title,url,identifier,sem_scholar_title,sem_scholar_abstract,fc_score
0,811az5,Firearm Injuries Drop 20 Percent When Gun Owne...,http://www.nejm.org/doi/full/10.1056/NEJMc1712773,DOI:10.1056/NEJMc1712773,Reduction in Firearm Injuries during NRA Annua...,Decline in Firearm Injuries during NRA Convent...,4
1,814dwi,Supplementation with probiotics during late pr...,http://journals.plos.org/plosmedicine/article?...,DOI:10.1371/journal.pmed.1002507,Diet during pregnancy and infancy and risk of ...,Background There is uncertainty about the infl...,5
2,815lr6,Undisclosed Conflicts of Interests among Biome...,https://www.ncbi.nlm.nih.gov/pubmed/29400625,PMID:29400625,Undisclosed conflicts of interest among biomed...,ABSTRACT Background: Textbooks are a formative...,5
3,817tfx,One more species of Tardigrade (famed for thei...,http://journals.plos.org/plosone/article?id=10...,DOI:10.1371/journal.pone.0192210,An integrative description of Macrobiotus shon...,Tardigrade research in Japan dates back over 1...,2
4,8183ly,There's a20% reduction in gun injuries during ...,http://www.nejm.org/doi/full/10.1056/NEJMc1712773,DOI:10.1056/NEJMc1712773,Reduction in Firearm Injuries during NRA Annua...,Decline in Firearm Injuries during NRA Convent...,4
...,...,...,...,...,...,...,...
5753,dpnj23,"Statements about building walls, deportation a...",https://journals.plos.org/plosone/article?id=1...,DOI:10.1371/journal.pone.0222837,Declared impact of the US President’s statemen...,"Statements about building walls, deportation a...",5
5754,dpnu7e,Many college students will uncritically accept...,https://journals.plos.org/plosone/article?id=1...,DOI:10.1371/journal.pone.0223736,When calculators lie: A demonstration of uncri...,Calculators are often unnecessary to solve rou...,4
5755,dpqhem,New method for making polymers with perfectly ...,https://pubs.acs.org/doi/10.1021/jacs.9b08240,DOI:10.1021/jacs.9b08240,Homogenous Synthesis of Monodisperse High Olig...,Whereas monodisperse polymers are ubiquitous i...,4
5756,dptk4o,Research Shows That Doing a Bad Job Wrapping P...,https://onlinelibrary.wiley.com/doi/epdf/10.10...,DOI:10.1002/jcpy.1140,Presentation Matters: The Effect of Wrapping N...,While gift-givers typically wrap gifts prior t...,2


In [94]:
df_out.to_csv(f"{file_name}_fc_scores.csv", index=False)